In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
from torch.optim import AdamW

from transformers import RobertaTokenizer

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

device = device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/asrs-aeroguard/01_df_train_val_test.pkl


'cuda'

In [2]:
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

In [3]:
filepath = fr'/kaggle/input/asrs-aeroguard/01_df_train_val_test.pkl'
df_train_val_test = pd.read_pickle(filepath)

df_train = df_train_val_test.query('TRAIN_VAL_TEST_SPLIT == "Train"')
print(f'{df_train.shape=}')
df_val = df_train_val_test.query('TRAIN_VAL_TEST_SPLIT == "Validation"')
print(f'{df_val.shape=}')
df_test = df_train_val_test.query('TRAIN_VAL_TEST_SPLIT == "Test"')
print(f'{df_test.shape=}')

df_train

df_train.shape=(38655, 114)
df_val.shape=(4295, 114)
df_test.shape=(4773, 114)


,ACN_NUM_ACN,TIME_DATE,TIME_1_LOCAL_TIME_OF_DAY,PLACE_LOCALE_REFERENCE,PLACE_1_STATE_REFERENCE,PLACE_2_RELATIVE_POSITION_ANGLE_RADIAL,PLACE_3_RELATIVE_POSITION_DISTANCE_NAUTICAL_MILES,PLACE_4_ALTITUDE_AGL_SINGLE_VALUE,PLACE_5_ALTITUDE_MSL_SINGLE_VALUE,ENVIRONMENT_FLIGHT_CONDITIONS,...,ASSESSMENTS_CONTRIBUTING_FACTORS_SITUATIONS,ASSESSMENTS_1_PRIMARY_PROBLEM,REPORT_1_NARRATIVE,REPORT_1_1_CALLBACK,REPORT_2_NARRATIVE,REPORT_2_1_CALLBACK,REPORT_1_2_SYNOPSIS,TRAIN_VAL_TEST_SPLIT,EVENT_RISK,EVENT_RISK_STR
0,1574675,201808,0601-1200,SNA.Airport,CA,NaN,NaN,NaN,5000.0,NA,...,Human Factors,Human Factors,SNA RNP-Z to Runway 20R. The FMC was properly ...,NA,We were cleared for the RNP RNAV Z 20R Approac...,NA,B737-700 flight crew reported failing to make ...,Train,2,Medium risk
1,1224894,201412,0601-1200,MSY.Airport,LA,NaN,NaN,1000.0,NaN,VMC,...,Human Factors,Human Factors,On base to final turn to runway 1 in MSY at ap...,NA,NA,NA,Captain reports sighting of a drone at 1;000 f...,Train,0,Low risk
2,1134202,201312,1201-1800,ZZZ.ARTCC,US,NaN,NaN,NaN,2600.0,IMC,...,Human Factors; Aircraft; Procedure; Weather,Aircraft,I climbed to my filed altitude of 5;000 FT; an...,NA,NA,NA,SR22 pilot became disoriented on approach in I...,Train,3,Moderately high risk
3,1222074,201411,1201-1800,CWA.Airport,WI,NaN,20.0,NaN,4000.0,NA,...,Aircraft,Aircraft,I had my pitot heat checked prior to winter an...,NA,NA,NA,BE58 pilot experiences pitot heat failure desc...,Train,2,Medium risk
4,1733019,202003,1801-2400,ZDV.ARTCC,CO,NaN,NaN,NaN,32000.0,NA,...,Airspace Structure; Weather,Weather,At 32000 ft. just north of PUB the aircraft ex...,NA,NA,NA,B737 First Officer reported unexpected moderat...,Train,3,Moderately high risk
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38650,1093770,201306,NA,MRY.Airport,CA,330.0,3.0,NaN,2500.0,IMC,...,Human Factors; Aircraft,Ambiguous,Maintain directional control during climbout t...,NA,NA,NA,C172 instructor pilot reports difficulties wit...,Train,2,Medium risk
38651,1580993,201809,1201-1800,DTW.Airport,MI,NaN,NaN,250.0,NaN,NA,...,Environment - Non Weather Related; Procedure,Procedure,Given takeoff clearance from 21R in DTW behind...,NA,NA,NA,CRJ-900 Captain reported encountering wake tur...,Train,2,Medium risk
38652,1438881,201704,NA,ZZZ.Airport,US,NaN,NaN,0.0,NaN,NA,...,Company Policy; Human Factors,Company Policy,I did not witness passenger boarding as I was ...,NA,NA,NA,Boeing 787 Flight Attendant reported an adult ...,Train,0,Low risk
38653,1614309,201901,0601-1200,ZZZ.ARTCC,US,NaN,NaN,NaN,45000.0,VMC,...,Aircraft; Human Factors,Aircraft,Extreme cold cockpit temperatures were experie...,NA,[Report narrative contained no additional info...,NA,EMB-505 flight crew reported the loss of tempe...,Train,2,Medium risk


In [4]:
def preprocess_inputs(df):
    df = df.copy()
    
    X = df['REPORT_1_NARRATIVE']
    y = df['EVENT_RISK']
    
    X = X.apply(lambda x: tokenizer.encode(x, padding='max_length', truncation=True))
    X = X.apply(pd.Series)
    
    return X, y

In [5]:
X_train, y_train = preprocess_inputs(df_train)
X_val, y_val = preprocess_inputs(df_val)
X_test, y_test = preprocess_inputs(df_test)

X_train = torch.tensor(X_train.values).to(device)
y_train = torch.tensor(y_train.values).long().to(device)
X_val = torch.tensor(X_val.values).to(device)
y_val = torch.tensor(y_val.values).long().to(device)
X_test = torch.tensor(X_test.values).to(device)
y_test = torch.tensor(y_test.values).long().to(device)

In [6]:
tokenizer.vocab_size

50265

In [7]:
class WordEmbeddingModel(nn.Module):
    def __init__(self, vocab_size, sequence_length, embedding_size, hidden_size, output_size):
        super(WordEmbeddingModel, self).__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_size)
        
        self.fcs = nn.ModuleList([
            nn.Linear(sequence_length * embedding_size, hidden_size),
#             nn.Linear(hidden_size, hidden_size),
#             nn.Linear(hidden_size, hidden_size),
#             nn.Linear(hidden_size, hidden_size),
            nn.Linear(hidden_size, output_size),
        ])
        
        self.leaky_relu = nn.LeakyReLU()
    
    def forward(self, x):
        embedded = self.embedding(x)
        flattened = torch.flatten(embedded, 1)
        
        out = flattened.to(device)  # Move flattened tensor to the desired device
        
        for fc in self.fcs:
            out = fc(out)
            out = self.leaky_relu(out)
    
        return out

In [14]:
vocab_size = tokenizer.vocab_size
embedding_size = 128
hidden_size = 128
output_size = 5
sequence_length = 512

learning_rate = 0.01
# REG_PARAM = 0.02
num_epochs = 100
batch_size = 1024

model = WordEmbeddingModel(vocab_size, sequence_length, embedding_size, hidden_size, output_size).to(device)
criterion = nn.CrossEntropyLoss()
# optimizer = AdamW(model.parameters(), lr=learning_rate, weight_decay=REG_PARAM)
optimizer = AdamW(model.parameters(), lr=learning_rate)

In [12]:
X_train = torch.cat((X_train, X_test))
y_train = torch.cat((y_train, y_test))

In [15]:
# Create data loaders for train, validation, and test sets
train_dataset = torch.utils.data.TensorDataset(X_train, y_train)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

val_dataset = torch.utils.data.TensorDataset(X_val, y_val)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# test_dataset = torch.utils.data.TensorDataset(X_test, y_test)
# test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(num_epochs):
    train_losses = []
    val_losses = []

    # Train
    model.train()
    for inputs, labels in train_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = model(inputs)

        loss = criterion(outputs, labels)
        train_losses.append(loss.item())

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Evaluate
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)
            val_loss += criterion(outputs, labels).item()
            val_losses.append(val_loss)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    # Report metrics
    train_loss = np.mean(train_losses)
    val_loss = np.mean(val_losses)
    val_accuracy = 100.0 * correct / total
    print(f'epoch={epoch+1:02d} | train_loss={train_loss:.5f} | val_loss={val_loss:.5f} | val_acc={val_accuracy:.2f}%')


epoch=01 | train_loss=14.45176 | val_loss=4.62049 | val_acc=34.90%
epoch=02 | train_loss=1.52195 | val_loss=4.56320 | val_acc=35.51%
epoch=03 | train_loss=1.47260 | val_loss=4.60199 | val_acc=33.04%
epoch=04 | train_loss=1.38811 | val_loss=4.50623 | val_acc=36.86%
epoch=05 | train_loss=1.27291 | val_loss=4.57475 | val_acc=36.55%
epoch=06 | train_loss=1.05328 | val_loss=4.74602 | val_acc=39.91%
epoch=07 | train_loss=0.84332 | val_loss=4.89880 | val_acc=42.79%
epoch=08 | train_loss=0.59767 | val_loss=5.30814 | val_acc=42.49%
epoch=09 | train_loss=0.41486 | val_loss=6.50513 | val_acc=38.07%
epoch=10 | train_loss=0.31276 | val_loss=6.65218 | val_acc=38.93%


KeyboardInterrupt: 

In [16]:
model.eval()
test_losses = []
correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs.to(device)
        labels.to(device)
        outputs = model(inputs)
        test_losses.append(criterion(outputs, labels).item())
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

# Print test metrics
test_loss = np.mean(test_losses)
test_accuracy = 100.0 * correct / total
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}%")

Test Loss: 8.5088, Test Accuracy: 33.12%
